# GP Manifolds

## Setup Notebook

Before running this interactive notebook, check the following parameters:
- `ENV`: Select your execution environment `"LOCAL"` (VS Code, Jupyter Notebook) or `"COLAB"` (Google Colab).
- `PLT_INTERACTIVE`: Select whether the plots should be interactive to allow zooming, panning and resizing. In general, this can stay activated.

In [ ]:
#@title { display-mode: "form" }

ENV = "LOCAL" #@param ["LOCAL", "COLAB"]
PLT_INTERACTIVE = True #@param {type:"boolean"}

# setup environment
LOCAL = "LOCAL"
COLAB = "COLAB"
if ENV == COLAB:
    %cd /content
    !git clone https://github.com/danielyxyang/active_reconstruction.git
    %cd active_reconstruction
    !git checkout gp_manifolds
    !git submodule update --init
    %pip install -q -r requirements.txt
    %pip install -q -r src/utils_ext/requirements.txt
    %cd src
elif ENV == LOCAL:
    %cd ../src

# setup interactive plots
if PLT_INTERACTIVE:
    if ENV == COLAB:
        from google.colab import output
        output.enable_custom_widget_manager()
    %matplotlib widget
else:
    %matplotlib inline

# ensure automatic reload of imported modules
%load_ext autoreload
%autoreload 2


### CUSTOM SETUP ###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Ellipse
# from mpl_toolkits.mplot3d.ax

from utils_ext.math import is_in_range
from utils_ext.plot import PlotSaver
from algorithms.gp import GaussianProcess, build_kernel_matern, build_kernel_matern_periodic, build_kernel_matern_periodic_approx, build_kernel_rbf, build_kernel_rbf_periodic

plt.ioff()
PlotSaver.set_interactive(PLT_INTERACTIVE, is_colab=ENV == COLAB)

COLOR_MEAN = "tab:blue"
COLOR_CONFIDENCE = "gray"
COLOR_LOWER = "darkgray"
COLOR_UPPER = "dimgray"


## GP Curves (xy-parameterization)

To model any kind of 2D object shapes, one has to parameterize the surface with a parametric closed curve
$$
f\colon [0,2\pi) \to \mathbb{R}^2, t \mapsto (x,y)
$$
mapping some parameter $t$ to the $(x,y)$ coordinate on the surface. The idea of **GP curves** is to model the mapping for each coordinate
$$
f_x\colon [0,2\pi) \to \mathbb{R}, t \mapsto x \\
f_y\colon [0,2\pi) \to \mathbb{R}, t \mapsto y
$$
with a periodic Gaussian process. The challenge is to translate the separate confidence boundaries for the x- and y-coordinate into a meaningful confidence boundary for the closed curve.

In [ ]:
#@title { display-mode: "form" }

def build_gp_xy(n=100, **kwargs):
    t = np.linspace(0, 2*np.pi, n)
    m_x = lambda t: 10*np.cos(t)
    m_y = lambda t: 10*np.sin(t)
    k_x = build_kernel_matern_periodic(**kwargs)
    k_y = build_kernel_matern_periodic(**kwargs)
    gp_x = GaussianProcess(m_x, k_x, x_eval=t)
    gp_y = GaussianProcess(m_y, k_y, x_eval=t)
    return gp_x, gp_y


def plot_gp_curves_xy(
    gp_x, gp_y, name,
    swap=False,
    show="tx,ty,xy",
    show_boundary=True, boundary_class="ellipse",
    show_region=True,
    show_patches=None, patch_class="ellipse",
    show_samples=0,
    show_data=None,
):
    plots = []
    
    show = show.split(",")

    # define 1D confidence boundaries for x and y coordinate
    lower_x, upper_x = gp_x.confidence_boundary()
    lower_y, upper_y = gp_y.confidence_boundary()
    if swap:
        # swap upper and lower boundaries 
        lower_x_mask = is_in_range(gp_x.x_eval, (np.pi/2, 3*np.pi/2), mod=2*np.pi)
        upper_x_mask = is_in_range(gp_x.x_eval, (np.pi/2, 3*np.pi/2), mod=2*np.pi)
        upper_x[upper_x_mask], lower_x[lower_x_mask] = lower_x[lower_x_mask], upper_x[upper_x_mask]
        
        lower_y_mask = is_in_range(gp_y.x_eval, (np.pi, 2*np.pi), mod=2*np.pi)
        upper_y_mask = is_in_range(gp_y.x_eval, (np.pi, 2*np.pi), mod=2*np.pi)
        upper_y[upper_y_mask], lower_y[lower_y_mask] = lower_y[lower_y_mask], upper_y[upper_y_mask]

    # define 2D confidence boundaries
    if boundary_class == "rectangle":
        lower_boundary = np.array([lower_x, lower_y])
        upper_boundary = np.array([upper_x, upper_y])
    elif boundary_class == "ellipse":
        # compute angle perpendicular to curvature
        mean_dx = np.gradient(gp_x.mean)
        mean_dy = np.gradient(gp_y.mean)
        uncertainty_angle = np.arctan2(mean_dy, mean_dx) - np.pi/2
        uncertainty_direction = np.array([np.cos(uncertainty_angle), np.sin(uncertainty_angle)])
        # compute "diameter" of ellipse perpendicular to curvature
        ellipse_a = (upper_x - lower_x)/2
        ellipse_b = (upper_y - lower_y)/2
        uncertainty = np.sqrt((ellipse_a*np.cos(uncertainty_angle))**2 + (ellipse_b*np.sin(uncertainty_angle))**2) # https://math.stackexchange.com/a/3098035
        # compute upper and lower boundary
        mean = np.array([gp_x.mean, gp_y.mean])
        lower_boundary = mean - uncertainty * uncertainty_direction
        upper_boundary = mean + uncertainty * uncertainty_direction
    
    # define patch generator
    if patch_class == "rectangle":
        patch_generator = lambda lower_x, lower_y, upper_x, upper_y: Rectangle(
            (lower_x, lower_y),
            upper_x - lower_x,
            upper_y - lower_y,
        )
    elif patch_class == "ellipse":
        patch_generator = lambda lower_x, lower_y, upper_x, upper_y: Ellipse(
            ((upper_x+lower_x)/2, (upper_y+lower_y)/2),
            upper_x - lower_x,
            upper_y - lower_y,
        )
    
    # define step size at which patches are plotted
    if show_patches is not None:
        if show_patches == "all":
            patch_step = 1
        else:
            patch_step = int(np.ceil(len(gp_x.x_eval) / show_patches))

    # define samples
    if show_samples > 0:
        np.random.seed(0)
        samples_x = gp_x.sample(n=show_samples)
        samples_y = gp_y.sample(n=show_samples)

    # create t-x-plot
    if "tx" in show:
        fig, axis = PlotSaver.create()
        axis.set(title="{}_x".format(name), xlabel="t", ylabel="x")
        axis.plot(gp_x.x_eval, gp_x.mean, color=COLOR_MEAN)
        if show_boundary:
            axis.plot(gp_x.x_eval, lower_x, linewidth=1, color=COLOR_LOWER)
            axis.plot(gp_x.x_eval, upper_x, linewidth=1, color=COLOR_UPPER)
        if show_region:
            axis.fill_between(gp_x.x_eval, lower_x, upper_x, linewidth=0, color=COLOR_CONFIDENCE, alpha=0.2)
        if show_patches is not None and show_patches != "all":
            axis.plot(gp_x.x_eval[::patch_step], gp_x.mean[::patch_step], "o", markersize=3, color=COLOR_MEAN)
            axis.plot(gp_x.x_eval[::patch_step], lower_x[::patch_step], "o", markersize=3, color=COLOR_LOWER)
            axis.plot(gp_x.x_eval[::patch_step], upper_x[::patch_step], "o", markersize=3, color=COLOR_UPPER)
        if show_samples > 0:
            for sample_x in samples_x:
                axis.plot(gp_x.x_eval, sample_x, linewidth=1)
        if show_data is not None:
            axis.plot(*show_data[[0,1]], "o", markersize=4, color="red")
        plots.append((fig, "{}_x".format(name)))

    # create t-y-plot
    if "ty" in show:
        fig, axis = PlotSaver.create()
        axis.set(title="{}_y".format(name), xlabel="t", ylabel="y")
        axis.plot(gp_y.x_eval, gp_y.mean, color=COLOR_MEAN)
        if show_boundary:
            axis.plot(gp_y.x_eval, lower_y, linewidth=1, color=COLOR_LOWER)
            axis.plot(gp_y.x_eval, upper_y, linewidth=1, color=COLOR_UPPER)
        if show_region:
            axis.fill_between(gp_y.x_eval, lower_y, upper_y, linewidth=0, color=COLOR_CONFIDENCE, alpha=0.2)
        if show_patches is not None and show_patches != "all":
            axis.plot(gp_y.x_eval[::patch_step], gp_y.mean[::patch_step], "o", markersize=3, color=COLOR_MEAN)
            axis.plot(gp_y.x_eval[::patch_step], lower_y[::patch_step], "o", markersize=3, color=COLOR_LOWER)
            axis.plot(gp_y.x_eval[::patch_step], upper_y[::patch_step], "o", markersize=3, color=COLOR_UPPER)
        if show_samples > 0:
            for sample_y in samples_y:
                axis.plot(gp_y.x_eval, sample_y, linewidth=1)
        if show_data is not None:
            axis.plot(*show_data[[0,2]], "o", markersize=4, color="red")
        plots.append((fig, "{}_y".format(name)))
    
    # create x-y-plot
    if "xy" in show:
        fig, axis = PlotSaver.create()
        axis.set(
            title=name,
            xlabel="x", ylabel="y",
            xlim=(-13,13), ylim=(-13,13),
            aspect="equal",
        )
        axis.plot(gp_x.mean, gp_y.mean, color=COLOR_MEAN)
        if show_boundary:
            axis.plot(*lower_boundary, linewidth=1, color=COLOR_LOWER)
            axis.plot(*upper_boundary, linewidth=1, color=COLOR_UPPER)
        if show_region:
            axis.fill(
                np.concatenate([lower_boundary[0], upper_boundary[0, ::-1]]),
                np.concatenate([lower_boundary[1], upper_boundary[1, ::-1]]),
                linewidth=0, color=COLOR_CONFIDENCE, alpha=0.2,
            )
        if show_patches is not None:
            patches = [
                patch_generator(lower_x, lower_y, upper_x, upper_y)
                for lower_x, lower_y, upper_x, upper_y
                in zip(lower_x[::patch_step], lower_y[::patch_step], upper_x[::patch_step], upper_y[::patch_step])
            ]
            axis.add_collection(PatchCollection(patches, linewidth=0, color="gray", alpha=0.2))
            if show_patches != "all":
                axis.plot(gp_x.mean[::patch_step], gp_y.mean[::patch_step], "o", markersize=3, color=COLOR_MEAN)
                axis.plot(*lower_boundary[:, ::patch_step], "o", markersize=3, color=COLOR_LOWER)
                axis.plot(*upper_boundary[:, ::patch_step], "o", markersize=3, color=COLOR_UPPER)
        if show_samples > 0:
            for sample_x, sample_y in zip(samples_x, samples_y):
                axis.plot(sample_x, sample_y, linewidth=1)
        if show_data is not None:
            axis.plot(*show_data[[1,2]], "o", markersize=4, color="red")
        plots.append((fig, "{}".format(name)))
    
    return plots


# define dataset
data_txy = np.array([
    [np.pi/4*0, 6, 0],
    [np.pi/4*1, 8, 1],
    [np.pi/4*1.5, 4, 3],
    # 
    [np.pi/4*2, 0, 4],
    [np.pi/4*2.5, -2, 2],
    [np.pi/4*3, -4, 2],
    [np.pi/4*3.5, -4, 4],
    # 
    [np.pi/4*4, -8, 0],
    # 
    [np.pi/4*6, 0, -3],
]).T

### Rectangle Uncertainty

The naive way is to define the lower/interior 2D confidence boundary for the curve in terms of the lower 1D confidence boundaries for the x- and y-coordinates and likewise for the upper/exterior 2D confidence boundary. Intuitively, this results into interpreting the uncertainty area of a some point on the curve as a rectangle with width corresponding to the uncertainty interval in the x-coordinate and height corresponding to the uncertainty interval in the y-coordinate. The 2D confidence boundaries are then defined as the bottom left corners (i.e. lower x, lower y) and top right corners (i.e. upper x, upper y), respectively.

In [ ]:
def plot():
    plots = []

    # define GPs for xy-coordinates
    gp_x, gp_y = build_gp_xy(l=0.8)

    # plot prior
    kwargs = dict(patch_class="rectangle", boundary_class="rectangle")
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior", **kwargs, show="tx,ty,xy", show_patches=8)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior", **kwargs, show="xy", show_patches="all", show_region=False)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior_samples", **kwargs, show="tx,ty,xy", show_boundary=False, show_samples=3)

    PlotSaver.finish(plots, relsize=0.4, ratio=1, ncols=7)

plot()

As one can see, this does not result into correct confidence boundaries for the 2D object shape. The insight is that whenever the x- or y-coordinate becomes negative, the upper 2D confidence boundary for the curve is defined in terms of the lower confidence boundaries for the 1D coordinates (i.e. the "more negative" boundary) and vice versa for the lower confidence boundary. Hence, it seems intuitive to regularize the prior and to swap the upper and lower 1D confidence boundaries. This boils down to the following 2D confidence boundary definitions in terms of the corners of the uncertainty rectangles:
- quadrant 1: lower = bottom left // upper = top right
- quadrant 2: lower = bottom **right** // upper = top **left**
- quadrant 3: lower = **top** right // upper = **bottom** left
- quadrant 4: lower = top **left** // upper = bottom **right**

In [ ]:
def plot():
    plots = []

    # define GPs for xy-coordinates
    gp_x, gp_y = build_gp_xy(l=0.8)

    # define regularizer
    regularity_x = np.array([
        [np.pi/2*1, 0],
        [np.pi/2*3, 0],
    ]).T
    regularity_y = np.array([
        [np.pi/2*0, 0],
        [np.pi/2*2, 0],
    ]).T
    
    # plot regularized prior
    gp_x.update(*regularity_x)
    gp_y.update(*regularity_y)
    kwargs = dict(patch_class="rectangle", boundary_class="rectangle", swap=True)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior_regularized", **kwargs, show="tx,ty,xy", show_patches=8)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior_regularized", **kwargs, show="xy", show_patches="all", show_region=False)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior_regularized_samples", **kwargs, show="tx,ty,xy", show_boundary=False, show_samples=3)

    # plot posterior
    gp_x.update(data_txy[0], data_txy[1])
    gp_y.update(data_txy[0], data_txy[2])
    kwargs["show_data"] = data_txy
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior", **kwargs, show="tx,ty,xy", show_patches=8)
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior", **kwargs, show="xy", show_patches="all", show_region=False)
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior_samples", **kwargs, show="tx,ty,xy", show_boundary=False, show_samples=3)

    PlotSaver.finish(plots, relsize=0.4, ratio=1, ncols=7)

plot()

One can observe that the uncertainty areas shrink to zero whenever the upper and lower 1D confidence boundaries are swapped, which is intuitively incorrect.

In addition, defining the uncertainty as a rectangle is probabilistically incorrect, since this corresponds to defining the probability of some point on the curve to be at $(x,y)$ as
$$
p(x,y) = \max(p(x),p(y))
$$
instead of
$$
p(x,y) = p(x)p(y)
$$
when assuming independence between x- and y-coordinates.

### Ellipsoid Uncertainty

By modeling the x- and y-coordinates with GPs, each of the x- and y-coordinates follow a Gaussian distribution. Assuming that the x- and y-coordinates are independent the distribution of a point $(x,y)$ on the curve is distributed with a 2D Gaussian distribution, where the uncertainty area is described by an ellipse aligned with the coordinate axes. The 2D confidence boundaries are now defined in terms of the points on the ellipses lying on the "diameter" which is orthogonal to the mean 2D curve. This also removes the need of swapping the upper and lower 1D confidence boundaries in a complicated manner.

In [ ]:
def plot():
    plots = []

    # define GPs for xy-coordinates
    gp_x, gp_y = build_gp_xy(l=0.8)
    
    # plot prior
    kwargs = dict(patch_class="ellipse", boundary_class="ellipse")
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior", **kwargs, show="tx,ty,xy", show_patches=8)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior", **kwargs, show="xy", show_patches="all", show_region=False)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior_samples", **kwargs, show="tx,ty,xy", show_boundary=False, show_samples=3)

    # plot posterior
    gp_x.update(data_txy[0], data_txy[1])
    gp_y.update(data_txy[0], data_txy[2])
    kwargs["show_data"] = data_txy
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior", **kwargs, show="tx,ty,xy", show_patches=8)
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior", **kwargs, show="xy", show_patches="all", show_region=False)
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior_samples", **kwargs, show="tx,ty,xy", show_boundary=False, show_samples=3)

    PlotSaver.finish(plots, relsize=0.4, ratio=1, ncols=7)

plot()

Let's try some GPs kernels with smaller lengthscale and larger standard deviation to increase the uncertainty area.

In [ ]:
def plot():
    plots = []

    # define GPs for xy-coordinates
    gp_x, gp_y = build_gp_xy(l=0.2, sigma=1.5, n=250)

    # plot prior
    kwargs = dict(patch_class="ellipse", boundary_class="ellipse")
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior", **kwargs, show="tx,ty,xy", show_patches=8)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior", **kwargs, show="xy", show_patches="all", show_region=False)
    plots += plot_gp_curves_xy(gp_x, gp_y, "prior_samples", **kwargs, show="tx,ty,xy", show_boundary=False, show_samples=3)

    # plot posterior
    gp_x.update(data_txy[0], data_txy[1])
    gp_y.update(data_txy[0], data_txy[2])
    kwargs["show_data"] = data_txy
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior", **kwargs, show="tx,ty,xy", show_patches=8)
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior", **kwargs, show="xy", show_patches="all", show_region=False)
    plots += plot_gp_curves_xy(gp_x, gp_y, "posterior_samples", **kwargs, show="tx,ty,xy", show_boundary=False, show_samples=3)

    PlotSaver.finish(plots, relsize=0.4, ratio=1, ncols=7)

plot()

One can observe that the above definiton of 2D confidence boundaries is still not perfect, since it exhibits self-intersections. However, it also raises the question whether an explicit definition of 2D confidence boundaries for the GP curve is actually required or whether the notion of some uncertainty measure in terms of the ellipse area is sufficient.

One can clearly see from the 2D curve samples, that it is still difficult and appears nearly impossible to encode the property of no self-intersections into this GP-based representation.

## GP Curves (angle-speed-parameterization)

An alternative idea is to parameterize 2D curves in terms of their angle and speed
$$
\alpha\colon [0,2\pi) \to \mathbb{R}, t \mapsto \alpha(t) \\
v\colon [0,2\pi) \to \mathbb{R}, t \mapsto v(t)
,
$$
which define the first derivatives in the x- and y-coordinates of the curve. The x- and y-coordinates are then obtained by integration with
$$
f_x(t) = \int_0^t \cos(\alpha(t)) \cdot v(t) dt \\
f_y(t) = \int_0^t \sin(\alpha(t)) \cdot v(t) dt
.
$$
The advantage is that one can use GPs to model the mapping from the parameter $t$ to the angle and speed, which provides a more intuitive notion of similarity between points on the 2D curve. The GP modeling the angle describes how likely the 2D curve deviates from its current curvature (i.e. how off the next points are located from the current direction), while the GP modeling the speed describes how fast the 2D curve progresses into the direction of the current curvature (i.e. how far away the next points are located).

In [ ]:
#@title { display-mode: "form" }

def anglespeed2xy(angle, speed):
    dx = np.cos(angle) * speed
    dy = np.sin(angle) * speed
    x = np.cumsum(dx)
    y = np.cumsum(dy)
    return np.array([x, y])

def plot_gp_curves_anglespeed(angle, speed, name):
    fig, ax = PlotSaver.create()
    ax.set(title=name, aspect="equal")
    ax.plot(*anglespeed2xy(angle, speed), "-o", markersize=2)
    return [(fig, name)]

def build_gp_anglespeed():
    t = np.linspace(0, 2*np.pi, 100)
    m_angle = lambda t: t
    m_speed = lambda t: 1
    k_angle = build_kernel_rbf_periodic(l=0.8)
    k_speed = build_kernel_rbf_periodic(l=0.8)
    gp_angle = GaussianProcess(m_angle, k_angle, x_eval=t)
    gp_speed = GaussianProcess(m_speed, k_speed, x_eval=t)
    return gp_angle, gp_speed

We first test the angle-speed parameterization.

In [ ]:
def plot():
    plots = []
    angle = np.linspace(0, 2*np.pi, 25)
    speed = np.linspace(0, 10, 25)
    plots += plot_gp_curves_anglespeed(angle, speed, "parameterization")
    PlotSaver.finish(plots, relsize=0.5)

plot()

Now we plot the 2D curve obtained from our mean angle and mean speed function.

In [ ]:
def plot():
    plots = []
    gp_angle, gp_speed = build_gp_anglespeed()
    plots += plot_gp_curves_anglespeed(gp_angle.mean, gp_speed.mean, "prior")
    PlotSaver.finish(plots, relsize=0.5)

plot()

Next, we plot the 2D curves obtained from some some samples for our angle and speed function.

In [ ]:
def plot():
    plots = []
    gp_angle, gp_speed = build_gp_anglespeed()

    np.random.seed(0)
    samples_angle = gp_angle.sample(n=3)
    samples_speed = gp_speed.sample(n=3)
    for sample_angle, sample_speed in zip(samples_angle, samples_speed):
        plots += plot_gp_curves_anglespeed(sample_angle, sample_speed, "prior_sample")
    
    PlotSaver.finish(plots, relsize=0.5, ncols=3)

plot()

We can observe that sampling a random speed function can lead to negative speed and the 2D curve can go backwards. In addition the object shape can be modeled with any kind of speed function by adjusting the angle function appropriately. Hence, we decided to replace the random speed functions with the fixed mean speed function. The next plots show 2D curves obtained from some samples for our angle function and the fixed mean function for the speed.

In [ ]:
def plot():
    plots = []
    gp_angle, gp_speed = build_gp_anglespeed()
    
    np.random.seed(0)
    samples_angle = gp_angle.sample(n=3)
    samples_speed = gp_speed.sample(n=3)
    for sample_angle, sample_speed in zip(samples_angle, samples_speed):
        plots += plot_gp_curves_anglespeed(sample_angle, gp_speed.mean, "prior_sample")
    
    PlotSaver.finish(plots, relsize=0.5, ncols=3)

plot()

Although this parameterization provides more intuitive understanding for the choice of GP kernels, it is difficult to enforce no self-intersections and, more severely, closed curves.

## GP Surfaces (xzy-parameterization on torus)

In the following, we generalize GP curves to GP surfaces by modeling the mapping from some 2D parameter $(t_1,t_2)$ to a 3D coordinate $(x,y,z)$ with three separate GPs each describing the mapping
$$
f_x\colon [0,2\pi]^2 \to \mathbb{R}, (t_1,t_2) \mapsto x \\
f_y\colon [0,2\pi]^2 \to \mathbb{R}, (t_1,t_2) \mapsto y \\
f_z\colon [0,2\pi]^2 \to \mathbb{R}, (t_1,t_2) \mapsto z
$$
on the "square domain" $[0,2\pi]^2$. 
By choosing 2D periodic GP kernels, which can be obtained by periodic summation
$$
k(x,x') = \sum_{k\in\mathbb{Z}^2} k(x + 2\pi k,x') \text{ with } x,x'\in\mathbb{R}^2
,
$$
one intuitively connects the opposite "edges" of the "square domain" which results into "torus domain". Mathematically speaken, the GP is defined on the cartesian product of two circles $S^1 \times S^1$ which is homeomorphic to a ring torus. Hence, such a parameterization defines genus one surfaces (i.e. surfaces with a single hole) and we define our prior surface naturally as a torus.

In [ ]:
#@title { display-mode: "form" }

def build_gp_xyz(n=50, **kwargs):
    t1 = np.linspace(0, 2*np.pi, n)
    t2 = np.linspace(0, 2*np.pi, n)
    grid = np.reshape(np.meshgrid(t1, t2), (2, n*n)).T
    r1 = 10
    r2 = 5
    m_x = lambda t1t2: (r1 + r2*np.cos(t1t2[:, 0])) * np.cos(t1t2[:, 1])
    m_y = lambda t1t2: (r1 + r2*np.cos(t1t2[:, 0])) * np.sin(t1t2[:, 1])
    m_z = lambda t1t2: r2 * np.sin(t1t2[:, 0])
    k_x = build_kernel_matern_periodic_approx(**kwargs)
    k_y = build_kernel_matern_periodic_approx(**kwargs)
    k_z = build_kernel_matern_periodic_approx(**kwargs)
    gp_x = GaussianProcess(m_x, k_x, x_eval=grid, n_dims=2)
    gp_y = GaussianProcess(m_y, k_y, x_eval=grid, n_dims=2)
    gp_z = GaussianProcess(m_z, k_z, x_eval=grid, n_dims=2)
    return gp_x, gp_y, gp_z


def plot_gp_surface_torus(gp_x, gp_y, gp_z, name, show="mean", show_data=None):
    plots = []

    t1, t2 = np.reshape(gp_x.x_eval.T, (2, 50, 50))
    if show == "mean":
        x = np.reshape(gp_x.mean, (50, 50))
        y = np.reshape(gp_y.mean, (50, 50))
        z = np.reshape(gp_z.mean, (50, 50))
    elif show == "sample":
        x = np.reshape(gp_x.sample(), (50, 50))
        y = np.reshape(gp_y.sample(), (50, 50))
        z = np.reshape(gp_z.sample(), (50, 50))
    
    # create t1-t2-x plot
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    ax.set(title="{}_x".format(name), xlabel="t1", ylabel="t2", zlabel="x")
    ax.plot_surface(t1, t2, x)
    if show_data is not None:
        ax.plot(*show_data[[0,1,2]], "o", markersize=4, color="red")
    plots.append((fig, "{}_x".format(name)))
    
    # create t1-t2-y plot
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    ax.set(title="{}_y".format(name), xlabel="t1", ylabel="t2", zlabel="y")
    ax.plot_surface(t1, t2, y)
    if show_data is not None:
        ax.plot(*show_data[[0,1,3]], "o", markersize=4, color="red")
    plots.append((fig, "{}_y".format(name)))
    
    # create t1-t2-z plot
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    ax.set(title="{}_z".format(name), xlabel="t1", ylabel="t2", zlabel="z")
    ax.plot_surface(t1, t2, z)
    if show_data is not None:
        ax.plot(*show_data[[0,1,4]], "o", markersize=4, color="red")
    plots.append((fig, "{}_z".format(name)))

    # create x-y-z-plot
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    ax.set(title=name, xlabel="x", ylabel="y", zlabel="z")
    ax.plot_surface(x, y, z)
    if show_data is not None:
        ax.plot(*show_data[[2,3,4]], "o", markersize=4, color="red")
    ax.set(aspect="equal")
    plots.append((fig, name))

    return plots

In [ ]:
def plot():
    plots = []
    
    gp_x, gp_y, gp_z = build_gp_xyz()
    plots += plot_gp_surface_torus(gp_x, gp_y, gp_z, "prior", show="mean")
    plots += plot_gp_surface_torus(gp_x, gp_y, gp_z, "prior_sample", show="sample")

    data_t1t2_xyz = np.array([
        [np.pi/2*0.0, np.pi/2*0.0, 25, 10,  0],
        [np.pi/2*1.0, np.pi/2*1.0,  0, 10, 20],
        [np.pi/2*0.0, np.pi/2*2.0,-20,-10, -20],
    ]).T
    gp_x.update(data_t1t2_xyz[0:2].T, data_t1t2_xyz[2])
    gp_y.update(data_t1t2_xyz[0:2].T, data_t1t2_xyz[3])
    gp_z.update(data_t1t2_xyz[0:2].T, data_t1t2_xyz[4])
    plots += plot_gp_surface_torus(gp_x, gp_y, gp_z, "posterior", show="mean", show_data=data_t1t2_xyz)
    plots += plot_gp_surface_torus(gp_x, gp_y, gp_z, "posterior_sample", show="sample", show_data=data_t1t2_xyz)

    PlotSaver.finish(plots, relsize=0.5, ratio=0.9, ncols=4)

plot()

## GP Surfaces (xyz-parameterization on sphere)